In [1]:
%run ~/relmapping/annot_cb/notebooks/annot__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [2]:
l_atac_stages = list(config['annot_cb']['atac_samples'].keys())
args = ''
for stage in l_atac_stages:
    fp1 = pf('annot_cb_atac_%(stage)s_rep1' % locals(), config['annot_cb']['atac_step'], config['annot_cb']['atac_suffix'], 'atac')
    fp2 = pf('annot_cb_atac_%(stage)s_rep2' % locals(), config['annot_cb']['atac_step'], config['annot_cb']['atac_suffix'], 'atac')
    args += '%(stage)s %(fp1)s %(fp2)s ' % locals()

!~/repos/yapc/yapc annot_cb/metrics_atac/atac_cb {args} --smoothing-window-width 150 --fixed-peak-halfwidth 75    

18-09-18 17:08:41 | numpy version: 1.12.1
18-09-18 17:08:41 | pandas version: 0.20.3
18-09-18 17:08:41 | pyBigWig.numpy flag: 1
18-09-18 17:08:41 | IDR version: 2.0.3
18-09-18 17:08:41 | Input samples:
condition      sample                                                 fp  is_bw
   wt_ya  wt_ya_rep1  atac/tg_se.bwa_se_CB4.rm_unmapped.rm_contigs_C...   True
   wt_ya  wt_ya_rep2  atac/tg_se.bwa_se_CB4.rm_unmapped.rm_contigs_C...   True

18-09-18 17:08:41 | Output prefix: annot_cb/metrics_atac/atac_cb
18-09-18 17:08:41 | [compute]	pooled coverage track: annot_cb/metrics_atac/atac_cb_coverage.bw
calculating mean of all input signal:
I 15455979
II 16627154
III 14578851
IV 17485439
V 19495157
X 21540570
18-09-18 17:10:03 | [compute]	d2smooth track: annot_cb/metrics_atac/atac_cb_d2smooth.bw
I 15455979
II 16627154
III 14578851
IV 17485439
V 19495157
X 21540570
18-09-18 17:11:43 | [compute]	raw concave regions: annot_cb/metrics_atac/atac_cb_peaksall.tsv
/mnt/home1/ahringer/jj374/repos/yapc/ya

In [3]:
df_sites = pd.read_csv('annot_cb/metrics_atac/atac_cb.tsv', sep='\t')
l_globalIDR = ['%(stage)s_globalIDR' % locals() for stage in config['annot_cb']['atac_samples'].keys() ]
df_sites['max_globalIDR'] = df_sites[l_globalIDR].max(axis=1)
for th in [0.001, 0.005, 0.01, 0.05]:
    th_globalIDR = -math.log(th, 10)
    df_sites_th = df_sites.query('max_globalIDR >= @th_globalIDR')
    print('%d accessible sites at IDR=%.3f' % (len(df_sites_th), th))
df_sites.head()

8545 accessible sites at IDR=0.001
11772 accessible sites at IDR=0.005
13218 accessible sites at IDR=0.010
17074 accessible sites at IDR=0.050


,chrom,concave_start,concave_end,mode,wt_ya_rep1_score,wt_ya_rep2_score,min_rank,start,end,wt_ya_globalIDR,max_globalIDR
0,I,667,816,737,26.783079,9.875055,65296.0,662,813,0.093113,0.093113
1,I,912,1093,994,40.654358,12.653080,35360.0,919,1070,0.240213,0.240213
2,I,2125,2303,2219,81.727592,2.554739,14766.0,2144,2295,0.197491,0.197491
3,I,2414,2553,2483,0.233191,57.030091,25957.0,2408,2559,0.093113,0.093113
4,I,2926,3119,3025,73.401780,36.811700,16486.0,2950,3101,0.582160,0.582160


In [5]:
th_globalIDR = -math.log(0.001, 10)
fp_ = 'annot_cb/accessible_sites_cb.tsv'
df_sites.query('max_globalIDR > @th_globalIDR').to_csv(fp_, header=True, index=False, sep='\t')
!wc -l {fp_}

8546 annot_cb/accessible_sites_cb.tsv
